In [1]:
%pip install --upgrade google-cloud-speech

     |████████████████████████████████| 122 kB 5.1 MB/s 
     |████████████████████████████████| 44 kB 2.2 MB/s 
     |████████████████████████████████| 514 kB 57.3 MB/s 
     |████████████████████████████████| 596 kB 46.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [1]:
%pip install --upgrade google-cloud-storage

     |████████████████████████████████| 105 kB 5.1 MB/s 
     |████████████████████████████████| 75 kB 4.0 MB/s 
     |████████████████████████████████| 95 kB 3.5 MB/s 
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 1.26.3
    Uninstalling google-api-core-1.26.3:
      Successfully uninstalled google-api-core-1.26.3
  Attempting uninstall: google-resumable-media
    Found existing installation: google-resumable-media 0.4.1
    Uninstalling google-resumable-media-0.4.1:
      Successfully uninstalled google-resumable-media-0.4.1
  Attempting uninstall: google-cloud-core
    Found existing installation: google-cloud-core 1.0.3
    Uninstalling google-cloud-core-1.0.3:
      Successfully uninstalled google-cloud-core-1.0.3
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 1.18.1
    Uninstalling google-cloud-storage-1.18.1:
      Successfully uninstalled google-cloud-storage-1.18.1
ERROR: pip's depe

In [10]:
# this is because I put the API json key on my google drive
# skip this if you use local env
from google.colab import drive
drive.mount('/content/gdrive')
#ensure the file is accessible
!ls /content/gdrive/'My Drive'/'Colab Notebooks'/temp


Mounted at /content/gdrive
calibrate_647_generated242_data.ipynb  generated242.pkl
evaluate_IDM.ipynb		       temp_speech.json
expert_traj_env_train.pkl


In [12]:
# this is because I put the API json key on my google drive
# refer to google tutorial to check how to save the key
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/content/gdrive/My Drive/Colab Notebooks/temp/temp_speech.json"
#ensure the path is set correctly
!echo $GOOGLE_APPLICATION_CREDENTIALS

/content/gdrive/My Drive/Colab Notebooks/temp/temp_speech.json


In [2]:
from google.colab import auth
from google.cloud import storage
auth.authenticate_user()
project_id = 'prac-259701'
!gcloud config set project {project_id}
!gsutil ls
bucket_name = 'sage_ff14'

Updated property [core/project].
gs://sage_ff14/


In [3]:
# test, check if connecting with the cloud storage
from google.cloud import storage
project_id = 'prac-259701'
bucket_name = 'sage_ff14'
def list_blobs(bucket_name):
    """Lists all the blobs in the bucket."""
# Note: Client.list_blobs requires at least package version 1.17.0.
    storage_client = storage.Client(project_id)
    blobs = storage_client.list_blobs(bucket_name)
    for blob in blobs:
      print(blob.name)
list_blobs(bucket_name)

converted-sage.wav
converted.wav


In [4]:
!pip install srt

  Created wheel for srt: filename=srt-3.5.0-py3-none-any.whl size=22182 sha256=a1d652a765ce6eb783095eddd964b24af852a77c828a64fcdea97b838d4d958a
  Stored in directory: /root/.cache/pip/wheels/0f/38/79/e611a1233132e42439d7d703623fffd637d52ebabd0ec83658
Successfully built srt


In [5]:
import srt
from google.cloud import speech

In [6]:
# parameters about the audio file
sample_rate_hertz = 44100
language_code = "en-US"
audio_channel_count = 2
encoding = 'LINEAR16'
out_file = "subtitle"
max_chars = 40

In [7]:
def long_running_recognize(uri):
    """
    Transcribe long audio file from Cloud Storage using asynchronous speech
    recognition

    Args:
      storage_uri URI for audio file in GCS, e.g. gs://[BUCKET]/[FILE]
    """

    # print("Transcribing {} ...".format(args.storage_uri))
    client = speech.SpeechClient()

    # Encoding of audio data sent.
    operation = client.long_running_recognize(
        config=
        {
            "enable_word_time_offsets": True,
            "enable_automatic_punctuation": True,
            "sample_rate_hertz": sample_rate_hertz,
            "language_code": language_code,
            "audio_channel_count": audio_channel_count,
            "encoding": encoding,
        },
        audio={"uri": storage_uri},
    )
    response = operation.result()

    subs = []

    for result in response.results:
        # First alternative is the most probable result
        # alternative = result.alternatives[0]
        # subs.append(alternative)
        # print(u"Transcript: {}".format(alternative.transcript))

        subs = break_sentences( subs, result.alternatives[0])

    print("Transcribing finished")
    return subs



def break_sentences(subs, alternative):
    firstword = True
    charcount = 0
    idx = len(subs) + 1
    content = ""

    for w in alternative.words:
        if firstword:
            # first word in sentence, record start time
            # start = w.start_time.ToTimedelta()
            start = w.start_time

        charcount += len(w.word)
        content += " " + w.word.strip()

        if ("." in w.word or "!" in w.word or "?" in w.word or
                charcount > max_chars or
                ("," in w.word and not firstword)):
            # break sentence at: . ! ? or line length exceeded
            # also break if , and not first word
            subs.append(srt.Subtitle(index=idx,
                                     start=start,
                                    #  end=w.end_time.ToTimedelta(),
                                     end=w.end_time,
                                     content=srt.make_legal_content(content)))
            firstword = True
            idx += 1
            content = ""
            charcount = 0
        else:
            firstword = False
    return subs


def write_srt(subs):
    srt_file = out_file + ".srt"
    print("Writing {} subtitles to: {}".format(language_code, srt_file))
    f = open(srt_file, 'w')
    f.writelines(srt.compose(subs))
    f.close()
    return


def write_txt(subs):
    txt_file = out_file + ".txt"
    print("Writing text to: {}".format(txt_file))
    f = open(txt_file, 'w')
    for s in subs:
        f.write(s.content.strip() + "\n")
    f.close()
    return


In [8]:
storage_uri = 'gs://' + 'sage_ff14' + '/' +'converted-sage.wav'
print(storage_uri)

gs://sage_ff14/converted-sage.wav


In [13]:
subs = long_running_recognize(storage_uri)

Transcribing finished


In [14]:
subs

[Subtitle(index=1, start=datetime.timedelta(seconds=3, microseconds=200000), end=datetime.timedelta(seconds=3, microseconds=900000), content=' Hey, everyone.', proprietary=''),
 Subtitle(index=2, start=datetime.timedelta(seconds=3, microseconds=900000), end=datetime.timedelta(seconds=4, microseconds=400000), content=' Mr.', proprietary=''),
 Subtitle(index=3, start=datetime.timedelta(seconds=4, microseconds=400000), end=datetime.timedelta(seconds=4, microseconds=900000), content=' Happy here.', proprietary=''),
 Subtitle(index=4, start=datetime.timedelta(seconds=4, microseconds=900000), end=datetime.timedelta(seconds=7, microseconds=100000), content=' And in this video will be doing a full breakdown on', proprietary=''),
 Subtitle(index=5, start=datetime.timedelta(seconds=7, microseconds=100000), end=datetime.timedelta(seconds=7, microseconds=900000), content=' the stage job,', proprietary=''),
 Subtitle(index=6, start=datetime.timedelta(seconds=7, microseconds=900000), end=datetime.ti

In [15]:
    write_srt(subs)
    write_txt(subs)

Writing en-US subtitles to: subtitle.srt
Writing text to: subtitle.txt
